In [1]:
# TU LIMPIEZA Y ANÁLISIS AQUÍ

import pandas as pd
import seaborn as sns
import numpy as np

tv_df = pd.read_csv('tvs.csv')

tv_df


,_id,created_by,first_air_date,genres,id,in_production,last_air_date,name,number_of_episodes,number_of_seasons,...,original_language,original_name,overview,popularity,poster_path,production_companies,status,tagline,vote_average,vote_count
0,6457d26c7b901e08b141869d,[],1996-10-09,"[{""id"":35,""name"":""Comedy"",""_id"":{""$oid"":""6457d...",13,True,2022-02-16,How do you like Wednesday?,568.0,4,...,ja,水曜どうでしょう,How do you like Wednesday? was a Japanese tele...,21.207,/i2MwS6U0XzD8ad6aS3HiTNKz8ov.jpg,[],Returning Series,NaN,9.200,2
1,6457d26c7b901e08b141867a,"[{""id"":19303,""name"":""Kevin Smith"",""_id"":{""$oid...",2000-05-31,"[{""id"":16,""name"":""Animation"",""_id"":{""$oid"":""64...",2,False,2002-12-22,Clerks,6.0,1,...,en,Clerks,The continuing adventures of store clerks Dant...,10.620,/xunXvzFlkf1GGgMkCySA9CCFumB.jpg,"[{""id"":1558,""name"":""Touchstone Television"",""lo...",Canceled,NaN,6.897,78
2,6457d26c7b901e08b141869f,[],2006-08-20,"[{""id"":35,""name"":""Comedy"",""_id"":{""$oid"":""6457d...",14,False,2006-11-11,Shuriken School,26.0,1,...,en,Shuriken School,Shuriken School is a British animated series t...,3.034,/khXHvYKAvGqjtnKoLVYZr8r5yKD.jpg,"[{""id"":75257,""name"":""Zinkia Entertainment"",""lo...",Ended,NaN,5.500,6
3,6457d26c7b901e08b14186a5,"[{""id"":1074129,""name"":""Fred Rogers"",""_id"":{""$o...",1968-02-19,"[{""id"":10762,""name"":""Kids"",""_id"":{""$oid"":""6457...",15,False,2001-09-07,Mister Rogers' Neighborhood,850.0,31,...,en,Mister Rogers' Neighborhood,Mister Rogers' Neighborhood is an American chi...,19.167,/4Gz46uJg6MFaM4xeblbVNSxCyTr.jpg,"[{""id"":16021,""name"":""WQED"",""logo_path"":""/2bjAi...",Ended,Won't you be my neighbor?,4.977,88
4,6457d26c7b901e08b14186ab,"[{""id"":1211910,""name"":""Richard R. Lee"",""_id"":{...",NaN,[],16,False,2005-09-09,Sidewalks: Video Nite,399.0,11,...,en,Sidewalks: Video Nite,Sidewalks: Video Nite is an American televisio...,2.910,NaN,[],Ended,NaN,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150797,645c00b1378e4a37f5092e21,[],2015-06-12,"[{""id"":99,""name"":""Documentary"",""_id"":{""$oid"":""...",226231,False,2015-06-13,北京之夜,4.0,1,...,zh,北京之夜,NaN,0.000,NaN,[],Ended,NaN,0.000,0
150798,645c00b1378e4a37f5092e23,[],2014-07-29,"[{""id"":99,""name"":""Documentary"",""_id"":{""$oid"":""...",226232,False,2014-09-02,北洋海军兴亡史,6.0,1,...,zh,北洋海军兴亡史,NaN,0.000,/rcNz0Amku0XWOC3LuLfRHiKYqS3.jpg,[],Ended,NaN,0.000,0
150799,645c00b1378e4a37f5092e25,[],2023-05-09,[],226233,True,2023-05-09,Turkey: Empire of Erdogan,1.0,1,...,en,Turkey: Empire of Erdogan,A few months after his brutal crackdown on pro...,0.000,/l7rcVP8Q5yY4M7O6rJSstV7eIin.jpg,[],Returning Series,NaN,0.000,0
150800,645c00b1378e4a37f5092e26,[],2023-01-24,[],226234,True,2023-01-24,Woooooo! Becoming Ric Flair,1.0,1,...,en,Woooooo! Becoming Ric Flair,Ric Flair Biography,0.000,NaN,[],Returning Series,NaN,0.000,0


In [2]:
tv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150802 entries, 0 to 150801
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _id                   150802 non-null  object 
 1   created_by            150802 non-null  object 
 2   first_air_date        119932 non-null  object 
 3   genres                150802 non-null  object 
 4   id                    150802 non-null  int64  
 5   in_production         150802 non-null  bool   
 6   last_air_date         122108 non-null  object 
 7   name                  150796 non-null  object 
 8   number_of_episodes    150168 non-null  float64
 9   number_of_seasons     150802 non-null  int64  
 10  origin_country        150802 non-null  object 
 11  original_language     150802 non-null  object 
 12  original_name         150796 non-null  object 
 13  overview              84623 non-null   object 
 14  popularity            150802 non-null  float64
 15  

In [3]:
tv_df.isnull().sum()

_id                          0
created_by                   0
first_air_date           30870
genres                       0
id                           0
in_production                0
last_air_date            28694
name                         6
number_of_episodes         634
number_of_seasons            0
origin_country               0
original_language            0
original_name                6
overview                 66179
popularity                   0
poster_path              55875
production_companies         0
status                       0
tagline                 146238
vote_average                 0
vote_count                   0
dtype: int64

In [4]:
tv_df.isna().sum()

_id                          0
created_by                   0
first_air_date           30870
genres                       0
id                           0
in_production                0
last_air_date            28694
name                         6
number_of_episodes         634
number_of_seasons            0
origin_country               0
original_language            0
original_name                6
overview                 66179
popularity                   0
poster_path              55875
production_companies         0
status                       0
tagline                 146238
vote_average                 0
vote_count                   0
dtype: int64

In [5]:
tv_df.shape[0]

150802

In [6]:
import cleaning as cl

# Aplicar la función de limpieza a lo largo de las filas y crear nuevas columnas con datos limpios
tv_df["new_genres"] = tv_df.apply(cl.limpiar_genero, axis=1)
tv_df["new_creator"] = tv_df.apply(cl.limpiar_creador, axis=1)
tv_df["new_prductora"] = tv_df.apply(cl.limpiar_productora, axis=1)

#borrar columnas que no interesan
cl.borrar_columna(tv_df,"poster_path")
cl.borrar_columna(tv_df,"tagline")
cl.borrar_columna(tv_df,"overview")

#he considerado que las filas con name == NAN no deberian entrar en el análisis
#no tiene sentido que haya series sin nombres y sin datos del numero de episodios

cl.limpiar_filas_nan(tv_df, "name")
cl.limpiar_filas_nan(tv_df, "number_of_episodes")
cl.limpiar_filas_nan(tv_df, "first_air_date")
cl.limpiar_filas_nan(tv_df, "last_air_date")


La columna 'poster_path' se ha eliminado del DataFrame.
La columna 'tagline' se ha eliminado del DataFrame.
La columna 'overview' se ha eliminado del DataFrame.


In [7]:
df_limpio = tv_df[["_id", "first_air_date", "id", "in_production", "last_air_date", "name", "number_of_episodes", "number_of_seasons","original_language", "popularity", "status", "vote_count", "new_genres", "new_creator", "new_prductora"]]

df_limpio.to_csv(r'clean_data.csv', index=False)

In [8]:
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119038 entries, 0 to 150801
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   _id                 119038 non-null  object 
 1   first_air_date      119038 non-null  object 
 2   id                  119038 non-null  int64  
 3   in_production       119038 non-null  bool   
 4   last_air_date       119038 non-null  object 
 5   name                119038 non-null  object 
 6   number_of_episodes  119038 non-null  float64
 7   number_of_seasons   119038 non-null  int64  
 8   original_language   119038 non-null  object 
 9   popularity          119038 non-null  float64
 10  status              119038 non-null  object 
 11  vote_count          119038 non-null  int64  
 12  new_genres          119038 non-null  object 
 13  new_creator         119038 non-null  object 
 14  new_prductora       119038 non-null  object 
dtypes: bool(1), float64(2), int64(3), 

In [9]:
df_limpio

,_id,first_air_date,id,in_production,last_air_date,name,number_of_episodes,number_of_seasons,original_language,popularity,status,vote_count,new_genres,new_creator,new_prductora
0,6457d26c7b901e08b141869d,1996-10-09,13,True,2022-02-16,How do you like Wednesday?,568.0,4,ja,21.207,Returning Series,2,Comedy,Sin creador especificado,Productora independiente
1,6457d26c7b901e08b141867a,2000-05-31,2,False,2002-12-22,Clerks,6.0,1,en,10.620,Canceled,78,"Animation, Comedy","Kevin Smith, Scott Mosier, David Mandel","Touchstone Television, View Askew Productions,..."
2,6457d26c7b901e08b141869f,2006-08-20,14,False,2006-11-11,Shuriken School,26.0,1,en,3.034,Ended,6,"Comedy, Animation, Action & Adventure",Sin creador especificado,"Zinkia Entertainment, Studio Xilam"
3,6457d26c7b901e08b14186a5,1968-02-19,15,False,2001-09-07,Mister Rogers' Neighborhood,850.0,31,en,19.167,Ended,88,Kids,Fred Rogers,"WQED, Small World Enterprises, Family Communic..."
6,6457d26c7b901e08b14186c9,1985-09-07,25,False,1985-11-30,Star Wars: Droids,13.0,1,en,11.592,Ended,66,"Kids, Animation, Action & Adventure, Sci-Fi & ...","George Lucas, Ben Burtt, Clive A. Smith, Raymo...","Lucasfilm Animation, Lucasfilm Ltd."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150797,645c00b1378e4a37f5092e21,2015-06-12,226231,False,2015-06-13,北京之夜,4.0,1,zh,0.000,Ended,0,Documentary,Sin creador especificado,Productora independiente
150798,645c00b1378e4a37f5092e23,2014-07-29,226232,False,2014-09-02,北洋海军兴亡史,6.0,1,zh,0.000,Ended,0,Documentary,Sin creador especificado,Productora independiente
150799,645c00b1378e4a37f5092e25,2023-05-09,226233,True,2023-05-09,Turkey: Empire of Erdogan,1.0,1,en,0.000,Returning Series,0,Sin género,Sin creador especificado,Productora independiente
150800,645c00b1378e4a37f5092e26,2023-01-24,226234,True,2023-01-24,Woooooo! Becoming Ric Flair,1.0,1,en,0.000,Returning Series,0,Sin género,Sin creador especificado,Productora independiente


In [10]:
df_limpio.isna().sum()

_id                   0
first_air_date        0
id                    0
in_production         0
last_air_date         0
name                  0
number_of_episodes    0
number_of_seasons     0
original_language     0
popularity            0
status                0
vote_count            0
new_genres            0
new_creator           0
new_prductora         0
dtype: int64

In [11]:
df_limpio['last_air_date'] = pd.to_datetime(df_limpio.last_air_date)
df_limpio['first_air_date'] = pd.to_datetime(df_limpio.first_air_date)

df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119038 entries, 0 to 150801
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   _id                 119038 non-null  object        
 1   first_air_date      119038 non-null  datetime64[ns]
 2   id                  119038 non-null  int64         
 3   in_production       119038 non-null  bool          
 4   last_air_date       119038 non-null  datetime64[ns]
 5   name                119038 non-null  object        
 6   number_of_episodes  119038 non-null  float64       
 7   number_of_seasons   119038 non-null  int64         
 8   original_language   119038 non-null  object        
 9   popularity          119038 non-null  float64       
 10  status              119038 non-null  object        
 11  vote_count          119038 non-null  int64         
 12  new_genres          119038 non-null  object        
 13  new_creator         119038 no

<ipython-input-11-a0cd2312b1d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['last_air_date'] = pd.to_datetime(df_limpio.last_air_date)
<ipython-input-11-a0cd2312b1d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['first_air_date'] = pd.to_datetime(df_limpio.first_air_date)


### PREGUNTAS

In [12]:
#1 ¿Cual es la serie con la duración más corta?

df_limpio.number_of_episodes.value_counts().sort_index()

0.0           14
1.0        17824
2.0         5480
3.0         5372
4.0         5872
           ...  
7973.0         1
8903.0         1
10456.0        1
15704.0        1
19925.0        1
Name: number_of_episodes, Length: 925, dtype: int64

In [13]:
#Hay algunas con 0 episodios y muchas con 1 episodio
df_limpio['name'][df_limpio.number_of_episodes==df_limpio.number_of_episodes.min()]

6902                               The Joey Bishop Show
40113                                      The IT Crowd
57858     Ich heiße Mikrobi - Lustige Weltraumabenteuer
66639                    The Adventures of Marco & Gina
83570                                Xuanzang's Journey
90182                                             Idiot
96473                       テイルズ オブ クレストリア―咎我ヲ背負いて彼は発つ―
106852               Akechi Kogoro vs. Kindaichi Kosuke
106856                                          ラーメン発見伝
109443                                           THE100
110624                                  Kudi Yedamaithe
114743                                 Makubes no 23 ji
117045                                          Alice 6
133811                                       W'jaa Trab
Name: name, dtype: object

In [14]:
#2. ¿Cual es la serie con la duración más larga?

df_limpio['name'][df_limpio.number_of_episodes==df_limpio.number_of_episodes.max()]

82482    Tagesschau
Name: name, dtype: object

In [15]:
#3. ¿Cual es la serie con más temporadas? ¿Y la que tiene más capítulos?

df_limpio[["name", "number_of_episodes", "number_of_seasons"]][df_limpio.number_of_seasons==df_limpio.number_of_seasons.max()]

,name,number_of_episodes,number_of_seasons
15769,Fuse,2518.0,345


In [16]:
#4. ¿Cual es el género más estrenado para cada mes y año?

df_limpio['new_genres_list'] = df_limpio['new_genres'].str.split(', ')

df_limpio.explode('new_genres_list').groupby(df_limpio['first_air_date'].dt.to_period('M'))['new_genres_list'].max()

# df_limpio[df_limpio['new_genres'] != 'Sin género'].explode('new_genres_list').groupby(df_limpio['first_air_date'].dt.to_period('M'))['new_genres_list'].max()

<ipython-input-16-f5527ae43a21>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['new_genres_list'] = df_limpio['new_genres'].str.split(', ')


first_air_date
1917-01       Documentary
1936-11        Sin género
1938-07        Sin género
1938-08             Drama
1939-03        Sin género
                ...      
2023-01    War & Politics
2023-02           Western
2023-03           Western
2023-04           Western
2023-05    War & Politics
Freq: M, Name: new_genres_list, Length: 927, dtype: object

In [17]:
#5. ¿Cual es la media de temporadas por género?

df_limpio.explode('new_genres_list').groupby('new_genres_list')['number_of_seasons'].mean().sort_values(ascending=False)

# df_limpio[df_limpio['new_genres'] != 'Sin género'].explode('new_genres_list').groupby('new_genres_list')['number_of_seasons'].mean().sort_values(ascending=False)

new_genres_list
News                  6.959207
Music                 5.071429
Talk                  4.203249
Reality               3.016417
Erotic                3.000000
Family                2.395528
Kids                  2.291189
History               2.214286
Soap                  2.113379
Western               2.105263
Comedy                2.094466
Musical               2.000000
Documentary           1.987255
Crime                 1.941995
Action & Adventure    1.781761
Mystery               1.708824
Animation             1.681517
Sci-Fi & Fantasy      1.600876
Drama                 1.556616
Sin género            1.501585
War & Politics        1.380826
Romance               1.194444
Name: number_of_seasons, dtype: float64

In [18]:
#6. Listado de los 10 creadores más prolíficos.

creadores_prolificos = df_limpio[df_limpio['new_creator'] != 'Sin creador especificado'].value_counts('new_creator').head(10)
creadores_prolificos

new_creator
Khmerzoon                          102
Shotaro Ishinomori                  64
Adrián Suar                         61
John de Mol                         47
Joseph Barbera, William Hanna       46
Yang Li-Hua                         44
R.J. Nuevas                         36
Carla Albuquerque, Beto Ribeiro     34
Na Young-seok                       32
Ekta Kapoor                         32
dtype: int64

In [19]:
#7. ¿Que género es el que más producen estos escritores?

df_limpio[df_limpio['new_creator'].isin(creadores_prolificos.index)].explode('new_genres_list').groupby(df_limpio['new_creator'])['new_genres_list'].max()

new_creator
Adrián Suar                          War & Politics
Carla Albuquerque, Beto Ribeiro      War & Politics
Ekta Kapoor                                    Soap
John de Mol                                    Talk
Joseph Barbera, William Hanna               Western
Khmerzoon                            War & Politics
Na Young-seok                                  Talk
R.J. Nuevas                        Sci-Fi & Fantasy
Shotaro Ishinomori                          Western
Yang Li-Hua                              Sin género
Name: new_genres_list, dtype: object

In [20]:
#8. ¿Cuanto duran de media *en el aire* las series? ¿Y por género?

df_limpio['duracion'] = (df_limpio['last_air_date'] - df_limpio['first_air_date']).dt.days

df_limpio[['duracion']].mean()

<ipython-input-20-6fe4e8be8b31>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['duracion'] = (df_limpio['last_air_date'] - df_limpio['first_air_date']).dt.days


duracion    473.600716
dtype: float64

In [21]:
df_limpio.explode('new_genres_list').groupby('new_genres_list')['duracion'].mean().sort_values(ascending=False)

new_genres_list
News                  2985.773893
Music                 1804.071429
Talk                  1434.814129
Erotic                1246.000000
Reality                904.356463
Kids                   819.404345
Family                 724.158676
History                721.119048
Comedy                 597.814089
Soap                   597.633787
Western                587.315789
Crime                  585.637385
Animation              570.664165
Action & Adventure     514.878985
Documentary            499.981743
Sci-Fi & Fantasy       454.473617
Mystery                421.162925
Musical                392.000000
Drama                  342.990924
Sin género             287.102789
War & Politics         283.143142
Romance                209.294444
Name: duracion, dtype: float64

### TRANSFORMACIONES

In [22]:
#1. Crea una nueva columna llamada 'media_episodios' que el número de episodios por temporada de cada serie.

df_limpio['media_episodios'] = df_limpio['number_of_episodes'] / df_limpio['number_of_seasons']

<ipython-input-22-059b6a5dda28>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['media_episodios'] = df_limpio['number_of_episodes'] / df_limpio['number_of_seasons']


In [23]:
# 2. Crea una nueva columna llamada 'habla_inglesa' que indique si la serie está en inglés o no.

df_limpio['habla_inglesa'] = (df_limpio['original_language'] == 'en')

<ipython-input-23-e58ecad4d66c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['habla_inglesa'] = (df_limpio['original_language'] == 'en')


In [24]:
#3. Crear una nueva columna llamada 'produccion_europea' que indique si la serie es de producción europea o no.

# Lista de códigos de idioma correspondientes a países europeos
codigos_europeos = ['en', 'fr', 'de', 'es', 'it', 'pt', 'nl', 'sv', 'da', 'fi', 'no', 'pl', 'ru', 'tr', 'el', 'hu']

# Crear la nueva columna 'produccion_europea'
df_limpio['produccion_europea'] = df_limpio['original_language'].isin(codigos_europeos)

<ipython-input-24-34a62d5fc865>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['produccion_europea'] = df_limpio['original_language'].isin(codigos_europeos)


In [25]:
#4. Crea una nueva columna llamada 'mes_cancelacion' que indique el mes en el que se canceló la serie.

df_limpio['mes_cancelacion'] = df_limpio['last_air_date'][df_limpio['status']=='Canceled'].dt.month